In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-10-03 02:33:50--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.08s   

2021-10-03 02:33:50 (10.4 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
# Load in data from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_DVD_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Video_DVD_v1_00.tsv.gz"), inferSchema=True, sep='\t')
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   27288431| R33UPQQUZQEM8|B005T4ND06|     400024643|Yoga for Movement...|       Video DVD|          5|            3|          3|   N|                Y|This was a gift f...|This was a gift f...| 2015-08-31|
|         US|   13722556|R3IKTNQQPD9662|B004EPZ070|     685335564|  Something Borrowed|       Video DVD|          5|    

Drop duplicates and incomplete rows

In [5]:
print(df.count())
df =df.dropna();
print(df.count())

5069140
5068421


In [6]:
print(df.count())
df=df.drop_duplicates()
print(df.count())

5068421
5068421


In [7]:
#examine the schema
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [9]:
#change format of review_date from string to date
from pyspark.sql.functions import *
from pyspark.sql.functions import col
from pyspark.sql.types import StringType,BooleanType,DateType
df =df.withColumn('review_date',col('review_date').cast(DateType()))

In [10]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)



Create Table Dataframes

In [11]:
review_id_df= df.select(['review_id','customer_id','product_id','product_parent','review_date'])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1005M0EAS6Q9U|   12077103|B00VUK505O|      69985058| 2015-07-21|
|R100AJQA6PHRS3|   34193335|B0000CG8HQ|     396111937| 2004-07-02|
|R100P4A9IY1O39|   26305542|B00GB0OV1C|     507977556| 2014-04-30|
|R100QA0LZ39480|    5874616|B000U1ZV6W|     427231257| 2015-01-08|
|R10119LK56IJO4|   50859438|B00005MP58|      22701133| 2013-01-14|
|R10122LZFC6IUV|   12325534|B004TJ1H3C|     961261650| 2015-01-16|
|R10168OWZ66QZO|   44723143|B00001PE4D|     104267530| 2001-03-21|
|R101DMT5D0HEWH|   52878930|B00LLQ2A2S|     735480698| 2015-01-04|
|R101E0NW9RIGX0|   35565191|B003QWVPT4|     339765617| 2014-10-20|
|R101OZ414P7ARI|    2624502|B00FXB8RRO|     737158350| 2014-12-23|
|R101R4FIHCUONZ|   42559842|B000PKG69O|     755015170| 2014-02-07|
|R101U37NQETQMP|   36403248|B00003CXWM|      67501200| 2013-01

In [12]:
products_df = df.select(['product_id','product_title'])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00VUK505O|Lazarus Effect, T...|
|B0000CG8HQ|    Aria and Friends|
|B00GB0OV1C|  Chipmunk Adventure|
|B000U1ZV6W|4 Film Favorites:...|
|B00005MP58|Pride and Prejudi...|
|B004TJ1H3C|SGU: Stargate Uni...|
|B00001PE4D|    Cruel Intentions|
|B00LLQ2A2S|How We Got to Now...|
|B003QWVPT4|Why Did I Get Mar...|
|B00FXB8RRO|Attack on Titan, ...|
|B000PKG69O|    Hudson Hawk (Ws)|
|B00003CXWM|       Heartbreakers|
|B00DS79GY8|Veggie Tales: Mer...|
|B001C8W7EQ|Burn Notice: Seas...|
|B000CNESU8|Network (Two-Disc...|
|0790740826|    On Deadly Ground|
|B00007G1X1|Dark Shadows DVD ...|
|B001BWD6YS|Motivate Your Min...|
|B000RO9PU0|   Griffin & Phoenix|
|B000ION5SC|Over The Hedge [2...|
+----------+--------------------+
only showing top 20 rows



In [13]:
print(products_df.count())
products_df=products_df.drop_duplicates()
print(products_df.count())

5068421
297902


In [14]:
df.select(['customer_id']).distinct().count()

2075812

In [15]:
customers_df=df.groupby('customer_id').agg(count('customer_id').alias('customer_count'))

In [16]:
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   25498831|             2|
|   47490390|             3|
|   32702184|            22|
|   10206054|            16|
|   42780745|             1|
|   23105709|             2|
|   36176703|            35|
|   28707214|             3|
|   45286909|           195|
|   32142680|            77|
|   50927426|             1|
|   19540657|             2|
|   15422036|             1|
|   45763189|             9|
|   38851882|            92|
|   10704478|             1|
|   35523976|            21|
|   14564825|            13|
|   35390414|             1|
|   49243158|           274|
+-----------+--------------+
only showing top 20 rows



In [17]:
vine_df = df.select(['review_id','star_rating','helpful_votes','total_votes','vine'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1005M0EAS6Q9U|          5|            0|          1|   N|
|R100AJQA6PHRS3|          1|           21|         29|   N|
|R100P4A9IY1O39|          1|            0|         10|   N|
|R100QA0LZ39480|          5|            0|          0|   N|
|R10119LK56IJO4|          5|            0|          0|   N|
|R10122LZFC6IUV|          5|            0|          0|   N|
|R10168OWZ66QZO|          5|            1|          1|   N|
|R101DMT5D0HEWH|          5|            1|          1|   N|
|R101E0NW9RIGX0|          4|            0|          0|   N|
|R101OZ414P7ARI|          5|            0|          0|   N|
|R101R4FIHCUONZ|          3|            1|          1|   N|
|R101U37NQETQMP|          5|            0|          0|   N|
|R101UN1Y26WGQ3|          4|            2|          6|   N|
|R1027YFOX85498|          4|            

Load Dataframes into Postgres SQL DB

In [18]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://myprostgredb.cdhbjqbvqosf.us-east-2.rds.amazonaws.com:5432/my_data_class_db"
config = {"user":"postgres", 
          "password": "postgres", 
          "driver":"org.postgresql.Driver"}

In [19]:
# Write DataFrame to review_id_table table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [28]:
# Write DataFrame to products table in RDS ignore if data already exists

products_df.write.jdbc(url=jdbc_url, table='products', mode='ignore', properties=config)

In [27]:
# Write DataFrame to customers table in RDS j mode "ignore" if data already exists

customers_df.write.jdbc(url=jdbc_url, table='customers', mode='ignore', properties=config)

In [22]:
# Write DataFrame to vine_table table in RDS

vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)